Connect on MongoDB

In [1]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.scientific_articles
articles_collection = db.articles

Get all articles

In [2]:
articles_data = list(articles_collection.find({"sentences": {"$exists": False}}))


Instanciate BART

In [3]:
from transformers import BartForConditionalGeneration, AutoTokenizer
import torch
from tqdm import tqdm
import math

model_ckpt = "sshleifer/distilbart-cnn-6-6"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BartForConditionalGeneration.from_pretrained(model_ckpt)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Knowledge Extraction

For each article, Tokenize, Extract and Update the corresponding document on MongoDB

In [ ]:
from bson import ObjectId

max_size = 1024

for article_json in tqdm(articles_data, desc="Elaborating all articles"):

    input_id = tokenizer(article_json['body'], padding='max_length', return_tensors='pt').to(device)
    n_splits = math.ceil(len(input_id[0])/max_size)
    if(n_splits<5 and n_splits>1):
        print(f"n splits: {n_splits}")
        token_splits = []
        doc_summary_splits = []
        doc_summary_sentences = []
        
        # for each split of document, tokenize it and append into token_splits
        for index in list(range(n_splits)):
            if(index != n_splits-1):
                print(str(index*max_size) + " - " + str((index+1)*max_size))
                token_splits.append({ "input_ids": torch.tensor(input_id['input_ids'][0][index*max_size:(index+1)*max_size]).unsqueeze(0), 
                                    "attention_mask": torch.tensor(input_id['attention_mask'][0][index*max_size:(index+1)*max_size]).unsqueeze(0)})
            else:
                print(str(index*max_size) + " - " + str(len(input_id[0])%max_size + index*max_size))
                token_splits.append({ "input_ids": torch.tensor(input_id['input_ids'][0][index*max_size:len(input_id[0])%max_size + index*max_size]).unsqueeze(0), 
                                "attention_mask": torch.tensor(input_id['attention_mask'][0][index*max_size:len(input_id[0])%max_size + index*max_size]).unsqueeze(0)})
        
        # for each token split, generate token array summary 
        for token_split in tqdm(token_splits, desc="generating summary..."):
            doc_summary_splits.append(model.generate(input_ids=token_split['input_ids'], 
                                attention_mask=token_split['attention_mask'],
                                max_length=96))
            
        
        # for each token summary of document, decode it into text
        for doc_summary_split in doc_summary_splits:
            extracted_summary = tokenizer.decode(doc_summary_split[0], skip_special_tokens=True, clean_up_tokenization_spaces=True) + '\n'
            if ("." in extracted_summary):
                doc_summary_sentences.append(".".join(extracted_summary.split(".")[0:-1]))
            else:
                doc_summary_sentences.append(extracted_summary)
        
        articles_collection.update_one({"_id": ObjectId(article_json['_id'])}, {"$set": {"sentences": doc_summary_sentences}})

In [26]:
from bson import ObjectId
articles_collection.update_one({"_id": ObjectId(article_json['_id'])}, {"$set": {"sentences": [""]}})